# Simple Agentic Framework system
generating cold sales outreach emails (?)  
뭐하는 거지?

## Setup
- Sendgrid: https://sendgrid.com/  
Twilio의 메시지 전송 website.
- Sendgrid에서 api key 작성  
SENDGRID_API_KEY=SG.
- sender authentication에서 이메일 검증 등록

In [ ]:
# sendgrid 모듈 등록
%pip install sendgrid

In [ ]:
# 이메일 테스트 함 해봐
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [ ]:
load_dotenv(override = True)

In [ ]:
# 이메일 보내기가 될라나?
# 요거 잘 안됨. 다음 장애 나오는데 해결이 안되네.
#      URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)>
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("dinokrap@gmail.com")
    to_email = To("g1.systemswteam@gmail.com")
    content = Content("text/plain", "This is very important test email using sendgrid")
    mail = Mail(from_email, to_email, "SendGrid Email from Agents", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

## resend.com
sendgrid로 메일 보내기가 잘 되지 않음. 대안으로 resend.com으로 다시 한번 해보기.  
dinokrap으로 signin하고. 보낼 도메인을 확인하라고 한다. 나는 일단 domain은 갖고 있지 않으므로, resend의 도메인으로 한다.

In [ ]:
# pip로 resend 모듈은 설치하고
%pip install resend

In [ ]:
import os
import resend
from dotenv import load_dotenv

# 언제나 그렇듯이 환경 변수 로드
load_dotenv(override=True)

resend.api_key = os.getenv("RESEND_API_KEY")
params: resend.Emails.SendParams = {
    "from": "Acme <onboarding@resend.dev>",
    "to": "dinokrap@gmail.com",
    "subject": "Hello from visual code using resend",
    "html": "<p>This is an email sending test from vscode. <strong>Do you see it?</strong></p>"
}

email = resend.Emails.send(params)
print(email)

## Resend는 메일 보내기 성공함.
해서 Resend로 본 코드 작성 시작.

In [ ]:
# 처음부터 다시
# import
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import resend
import os
import asyncio

In [ ]:
# 언제나 그렇듯이
load_dotenv(override = True)

In [ ]:
# 이메일 보내기 테스트
def send_email():
    resend.api_key = os.getenv("RESEND_API_KEY")
    params: resend.Emails.SendParams = {
    "from": "Acme <onboarding@resend.dev>",
    "to": "dinokrap@gmail.com",
    "subject": "Hello from visual code using resend",
    "html": "<p>This is an email sending test from vscode. <strong>Do you see it?</strong></p>"
    }
    email = resend.Emails.send(params)

send_email()

In [ ]:
# Agent workflow
# 각 agent에 전달할 instruction 작성하고,
instruction1 = "You are a sales agent working for ComlAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for aduits, powered by AI. \
    You write professional, serious cold emails."
instruction2 = "You are a humorous, engaing sales agent working for CompAI, \
    a company that provides a Saas tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write witty, engaging cold emails that are likely to get a response."
instruction3 = "You are busy sales agent working for ComlAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write concise, to the point cold emails."

In [ ]:
# 3개의 각 agent 인스턴스를 생성
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions=instruction1,
    model="gpt-4o-mini"
)
sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instruction2,
    model="gpt-4o-mini"
)
sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instruction3,
    model="gpt-4o-mini"
)

In [ ]:
# 테스트로 agent #1을 실행하는데,
# run_streamed명령으로 생성하는 족족 결과를 실시간으로 받는다.
# 그래서 async for를 사용하고, stream_events를 실행해서 LLM이 만드는 모든 이벤트를 받는다.
result = Runner.run_streamed(sales_agent1, input="write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
# 이번에는 3개 agent를 한번에 실행하고,리턴 받는다.
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.Run(sales_agent2, message),
        Runner.Run(sales_agent3, message)
    )

output = [result.final_output for result in results]

for output in output for result in results